# 第五次作业

本次作业我们强化对GAE和VGAE的实践。具体地，我们用它们来完成（无监督）节点分类任务。

## 0. 课后习题

1. 图的自编码器和图卷积神经网络的区别是什么？
2. 图的变分自编码器比起图的自编码器的优点是什么？

## 1. 定义GAE和VGAE
首先请同学们定义两个类：GAE和VGAE。

In [2]:
class GCNEncoder(torch.nn.Module):
    """GCN组成的编码器"""
    def __init__(self, in_channels, out_channels):
        pass

class InnerProductDecoder(torch.nn.Module):
    def __init__(self):
        pass

class GAE(torch.nn.Module):
    """图自编码器。
    """
    def __init__(self, encoder, decoder=None):
        super().__init__()
        pass

In [3]:
class VariationalGCNEncoder(torch.nn.Module):

    def __init__(self, in_channels, out_channels):
        pass
    
class VGAE(GAE): 
    """变分自编码器。继承自GAE这个类，可以使用GAE里面定义的函数。
    """
    
    def __init__(self, encoder, decoder=None):
        super().__init__(encoder, decoder)
        pass

## 2. 加载数据集

本次作业我们使用Cora数据集。

In [4]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='./data', name='Cora')# 将数据保存在data文件夹下
data = dataset[0]

##  3. 训练模型

这部分请同学们自由发挥。

In [5]:
### 请大家完成下面这个测试函数 ###
def train(model, data, epochs=1000, variational=False):
    """训练模型
    
    参数说明
    ----
    model: 模型
    data: 数据
    epochs: 训练的周期
    variational: 是否训练VGAE。如果为True，则计算损失函数的时候需要加上kl loss
    """
    return 

### 3.1 GAE

In [6]:
in_channels, out_channels = dataset.num_features, 16
device = 'cpu' # device = 'cuda'
 
model = 

train(model, data, epochs=1000, variational=False)

Epoch: 000, Loss_train: 1.3696
Epoch: 100, Loss_train: 0.8451
Epoch: 200, Loss_train: 0.8140
Epoch: 300, Loss_train: 0.8023
Epoch: 400, Loss_train: 0.7899
Epoch: 500, Loss_train: 0.7936
Epoch: 600, Loss_train: 0.7933
Epoch: 700, Loss_train: 0.8009
Epoch: 800, Loss_train: 0.7886
Epoch: 900, Loss_train: 0.7884


### 3.2 VGAE

In [8]:
in_channels, out_channels = dataset.num_features, 16
device = 'cpu' # device = 'cuda'

model = 
model = model.to(device)

train(model, data, epochs=1000, variational=True)

Epoch: 000, Loss_train: 3.3813
Epoch: 100, Loss_train: 0.8639
Epoch: 200, Loss_train: 0.8309
Epoch: 300, Loss_train: 0.8261
Epoch: 400, Loss_train: 0.8189
Epoch: 500, Loss_train: 0.8189
Epoch: 600, Loss_train: 0.8133
Epoch: 700, Loss_train: 0.8076
Epoch: 800, Loss_train: 0.8179
Epoch: 900, Loss_train: 0.8103


## 4. 测试模型

训练一个线性模型（比如逻辑回归模型）来预测节点的标签，并输出预测准确率。

In [10]:
### 请大家完成下面这个测试函数 ###
def evaluate_node_classification(embedding_matrix, labels, train_mask, 
                                 test_mask, normalize_embedding=True, max_iter=1000):
        
    """训练一个线性模型（比如逻辑回归模型）来预测节点的标签
    
    返回值说明
    ----
    preds: 模型预测的标签
    test_acc: 模型预测的准确率
    """
    return preds, test_acc

In [11]:
preds, test_acc = evaluate_node_classification(embedding_gae, data.y, data.train_mask, data.test_mask)
print('GAE Test Acc: %.4f' % test_acc)

GAE Test Acc: 0.7120


In [12]:
preds, test_acc = evaluate_node_classification(embedding_vgae, data.y, data.train_mask, data.test_mask)
print('VGAE Test Acc: %.4f' % test_acc)

VGAE Test Acc: 0.6760


到这里本次作业就结束了。这次的任务其实是无监督的节点分类问题。可以看到，我们会使用一些和前面DeepWalk作业中相似的代码。同学可以讲DeepWalk和GAE/VGAE的结果做一个比较，看看谁在这个任务上效果更好。